In [1]:
# Limpieza de datos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# función para cargar los datos dada la dirección

def cargar_datos(direccion):
    df_list = []
    year = direccion.split('/')[-2]
    if year == '2024':
        lim = 8
    else:
        lim = 13
    
    for file in range(1, lim):
        print(f"Cargando archivo {file} de {lim-1}")
        if file < 10:
            file_path = direccion + year + "_" + f'0{file}.csv'
        else:
            file_path = direccion + year + "_" + f'{file}.csv'
        try:
            df = pd.read_csv(file_path, low_memory=False, encoding='utf-8')
        except UnicodeDecodeError:
            print(f"Problema de codificación en {file_path}, intentando con 'latin1'")
            df = pd.read_csv(file_path, low_memory=False, encoding='latin1')
        
        df_list.append(df)

    return pd.concat(df_list, ignore_index=True)

In [3]:
def calcular_tiempo_viaje(df):
    # Intentar convertir las columnas de Fecha y Hora en datetime, eliminando errores
    try:
        df['Fecha_Hora_Retiro'] = pd.to_datetime(df['Fecha_Retiro'] + ' ' + df['Hora_Retiro'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
        df['Fecha_Hora_Arribo'] = pd.to_datetime(df['Fecha_Arribo'] + ' ' + df['Hora_Arribo'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
    except Exception as e:
        print("Error al convertir fechas y horas:", e)
    
    # Verificar si las columnas 'Fecha_Hora_Retiro' y 'Fecha_Hora_Arribo' existen
    if 'Fecha_Hora_Retiro' not in df.columns or 'Fecha_Hora_Arribo' not in df.columns:
        print("Una o ambas columnas de fecha y hora no se crearon correctamente.")
        return df

    # Eliminar filas con datos inválidos (NaT) en las columnas de fechas
    df = df.dropna(subset=['Fecha_Hora_Retiro', 'Fecha_Hora_Arribo'])

    # Verificar si hay datos después de eliminar NaT
    if df.empty:
        print("Todas las filas fueron eliminadas por datos inválidos.")
        return df

    # Calcular la diferencia en minutos usando .loc para evitar el warning
    df.loc[:, 'Tiempo_viaje'] = (df['Fecha_Hora_Arribo'] - df['Fecha_Hora_Retiro']).dt.total_seconds() / 60

    # Filtrar los viajes que duren más de 90 minutos
    df = df[df['Tiempo_viaje'] <= 90].copy()

    # Eliminar las columnas temporales
    df = df.drop(columns=['Fecha_Hora_Retiro', 'Fecha_Hora_Arribo'])

    return df

In [7]:
def calcular_tiempo_viaje2(df):
    # Intentar convertir las columnas Inicio_del_viaje y Fin_del_viaje a datetime, eliminando errores
    try:
        df['Inicio_del_viaje'] = pd.to_datetime(df['Inicio_del_viaje'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        df['Fin_del_viaje'] = pd.to_datetime(df['Fin_del_viaje'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    except Exception as e:
        print("Error al convertir las columnas de fecha y hora:", e)
    
    # Eliminar filas con datos inválidos (NaT) en las columnas de fechas
    df = df.dropna(subset=['Inicio_del_viaje', 'Fin_del_viaje'])

    # Verificar si hay datos después de eliminar NaT
    if df.empty:
        print("Todas las filas fueron eliminadas por datos inválidos.")
        return df

    # Calcular la diferencia en minutos usando .loc para evitar el warning
    df.loc[:, 'Tiempo_viaje'] = (df['Fin_del_viaje'] - df['Inicio_del_viaje']).dt.total_seconds() / 60

    # Filtrar los viajes que duren más de 30 minutos
    df = df[df['Tiempo_viaje'] <= 90].copy()

    # Eliminar las columnas temporales
    #df = df.drop(columns=['Inicio_del_viaje', 'Fin_del_viaje'])

    return df


In [11]:
def calcular_tiempo_viaje2(df):
    # Función auxiliar para agregar ':00' si el formato de la hora es 'hh:mm'
    def agregar_segundos(hora):
        # Se separa la fecha de la hora
        partes = hora.split(" ")
        if len(partes) == 2:  # Asegurarse de que la cadena contenga una fecha y una hora
            fecha, hora_valor = partes
            # Si la hora está en formato 'h:mm' o 'hh:mm'
            if len(hora_valor) == 4:  # Caso 'h:mm'
                return f"{fecha} 0{hora_valor}:00"
            elif len(hora_valor) == 5:  # Caso 'hh:mm'
                return f"{fecha} {hora_valor}:00"
        return hora  # Devolver la cadena original si no cumple con los formatos

    # Intentar convertir las columnas Inicio_del_viaje y Fin_del_viaje a datetime, eliminando errores
    try:
        # Aplicar la función para agregar segundos donde sea necesario
        df['Inicio_del_viaje'] = df['Inicio_del_viaje'].apply(agregar_segundos)
        df['Fin_del_viaje'] = df['Fin_del_viaje'].apply(agregar_segundos)
        
        df['Inicio_del_viaje'] = pd.to_datetime(df['Inicio_del_viaje'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        df['Fin_del_viaje'] = pd.to_datetime(df['Fin_del_viaje'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    except Exception as e:
        print("Error al convertir las columnas de fecha y hora:", e)
    
    # Eliminar filas con datos inválidos (NaT) en las columnas de fechas
    df = df.dropna(subset=['Inicio_del_viaje', 'Fin_del_viaje'])

    # Verificar si hay datos después de eliminar NaT
    if df.empty:
        print("Todas las filas fueron eliminadas por datos inválidos.")
        return df

    # Calcular la diferencia en minutos usando .loc para evitar el warning
    df.loc[:, 'Tiempo_viaje'] = (df['Fin_del_viaje'] - df['Inicio_del_viaje']).dt.total_seconds() / 60

    # Filtrar los viajes que duren más de 90 minutos
    df = df[df['Tiempo_viaje'] <= 90].copy()

    # Eliminar las columnas temporales si no se necesitan más
    #df = df.drop(columns=['Inicio_del_viaje', 'Fin_del_viaje'])

    return df


In [13]:
def calcular_tiempo_viaje2(df):
    # Función auxiliar para agregar ':00' si el formato de la hora es 'h:mm' o 'hh:mm' y para manejar fechas en formato 'dd/mm/yyyy'
    def ajustar_formato(fecha_hora):
        partes = fecha_hora.split(" ")
        if len(partes) == 2:  # Asegurarse de que la cadena contenga una fecha y una hora
            fecha, hora_valor = partes
            
            # Comprobar si la fecha está en formato 'dd/mm/yyyy' y convertir a 'yyyy-mm-dd'
            if "/" in fecha:
                try:
                    # Convertir de 'dd/mm/yyyy' a 'yyyy-mm-dd'
                    dia, mes, año = fecha.split("/")
                    fecha = f"{año}-{mes}-{dia}"
                except ValueError:
                    # Si ocurre un error, devolver la cadena original
                    return fecha_hora
            
            # Ajustar la hora agregando ':00' si es necesario
            if len(hora_valor) == 4:  # Caso 'h:mm'
                return f"{fecha} 0{hora_valor}:00"
            elif len(hora_valor) == 5:  # Caso 'hh:mm'
                return f"{fecha} {hora_valor}:00"
        
        return fecha_hora  # Devolver la cadena original si no cumple con los formatos

    # Intentar convertir las columnas Inicio_del_viaje y Fin_del_viaje a datetime, eliminando errores
    try:
        # Aplicar la función para ajustar el formato de fecha y hora
        df['Inicio_del_viaje'] = df['Inicio_del_viaje'].apply(ajustar_formato)
        df['Fin_del_viaje'] = df['Fin_del_viaje'].apply(ajustar_formato)
        
        df['Inicio_del_viaje'] = pd.to_datetime(df['Inicio_del_viaje'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        df['Fin_del_viaje'] = pd.to_datetime(df['Fin_del_viaje'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    except Exception as e:
        print("Error al convertir las columnas de fecha y hora:", e)
    
    # Eliminar filas con datos inválidos (NaT) en las columnas de fechas
    df = df.dropna(subset=['Inicio_del_viaje', 'Fin_del_viaje'])

    # Verificar si hay datos después de eliminar NaT
    if df.empty:
        print("Todas las filas fueron eliminadas por datos inválidos.")
        return df

    # Calcular la diferencia en minutos usando .loc para evitar el warning
    df.loc[:, 'Tiempo_viaje'] = (df['Fin_del_viaje'] - df['Inicio_del_viaje']).dt.total_seconds() / 60

    # Filtrar los viajes que duren más de 30 minutos
    df = df[df['Tiempo_viaje'] <= 90].copy()

    # Eliminar las columnas temporales si no se necesitan más
    #df = df.drop(columns=['Inicio_del_viaje', 'Fin_del_viaje'])

    return df


In [5]:
# Vamos a cargar y limpiar todos los datos disponibles de ecobici

direccion = 'ecobici/'

years = ["2022"]

for year in years:
    df = cargar_datos(direccion + year + '/')
    if 'Unnamed: 9' in df.columns:
        df = df.drop(columns='Unnamed: 9')
    df = calcular_tiempo_viaje(df)
    df.to_csv(direccion + year + '.csv', index=False)
    del df
    print(f'Los datos de {year} han sido limpiados y guardados')

Cargando archivo 1 de 12
Cargando archivo 2 de 12
Cargando archivo 3 de 12
Cargando archivo 4 de 12
Cargando archivo 5 de 12
Cargando archivo 6 de 12
Cargando archivo 7 de 12
Cargando archivo 8 de 12
Cargando archivo 9 de 12
Cargando archivo 10 de 12
Cargando archivo 11 de 12
Cargando archivo 12 de 12


/tmp/ipykernel_29063/744448987.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Tiempo_viaje'] = (df['Fecha_Hora_Arribo'] - df['Fecha_Hora_Retiro']).dt.total_seconds() / 60


Los datos de 2022 han sido limpiados y guardados


In [14]:
# ahora con los datos de mibici

direccion = 'mibici/'

years = ["2023"]

for year in years:
    df = cargar_datos(direccion + year + '/')
    if 'Unnamed: 9' in df.columns:
        df = df.drop(columns='Unnamed: 9')
    df = calcular_tiempo_viaje2(df)
    df.to_csv(direccion + year + '.csv', index=False)
    del df
    print(f'Los datos de {year} han sido limpiados y guardados')

Cargando archivo 1 de 12
Problema de codificación en mibici/2023/2023_01.csv, intentando con 'latin1'
Cargando archivo 2 de 12
Problema de codificación en mibici/2023/2023_02.csv, intentando con 'latin1'
Cargando archivo 3 de 12
Problema de codificación en mibici/2023/2023_03.csv, intentando con 'latin1'
Cargando archivo 4 de 12
Problema de codificación en mibici/2023/2023_04.csv, intentando con 'latin1'
Cargando archivo 5 de 12
Problema de codificación en mibici/2023/2023_05.csv, intentando con 'latin1'
Cargando archivo 6 de 12
Problema de codificación en mibici/2023/2023_06.csv, intentando con 'latin1'
Cargando archivo 7 de 12
Problema de codificación en mibici/2023/2023_07.csv, intentando con 'latin1'
Cargando archivo 8 de 12
Problema de codificación en mibici/2023/2023_08.csv, intentando con 'latin1'
Cargando archivo 9 de 12
Problema de codificación en mibici/2023/2023_09.csv, intentando con 'latin1'
Cargando archivo 10 de 12
Problema de codificación en mibici/2023/2023_10.csv, int